In [0]:
from delta.tables import DeltaTable
from pyspark.sql import SparkSession

In [0]:
def merge_delta_table(df_join, tabela_destino):
    """
    Realiza MERGE (upsert) de um DataFrame em uma tabela Delta Lake.

    Regras de chaves:
    -----------------
    - Para 'workspace.bronze_etl.ipca' ou 'workspace.bronze_etl.boi_gordo': usa as colunas 'data', 'valor'
    - Para demais tabelas: usa as colunas 'data', 'ipca', 'boi_gordo'

    Parâmetros:
    -----------
    df_join : DataFrame
        DataFrame com os dados a serem inseridos/atualizados.
    tabela_destino : str
        Nome completo da tabela Delta (ex: 'workspace.bronze_etl.ipca' ou 'workspace.gold_etl.insights').

    Retorno:
    --------
    None
    """

    if df_join is not None and df_join.limit(1).count() > 0:
        total = df_join.count()
        print(f"✅ Total de registros carregados: {total}")

        if spark.catalog.tableExists(tabela_destino):
            print(f"📦 Tabela {tabela_destino} já existe — atualizando dados...")

            delta_table = DeltaTable.forName(spark, tabela_destino)

            # 🔑 Define a condição de merge conforme a tabela
            if tabela_destino in [
                "workspace.bronze_etl.ipca",
                "workspace.bronze_etl.boi_gordo"
            ]:
                condicao_merge = "t.data = s.data AND t.valor = s.valor"
                print("🔑 Usando chaves: data, valor")
            else:
                condicao_merge = (
                    "t.data = s.data "
                    "AND t.ipca = s.ipca "
                    "AND t.boi_gordo = s.boi_gordo"
                )
                print("🔑 Usando chaves: data, ipca, boi_gordo")

            # 🚀 Executa o MERGE
            (
                delta_table.alias("t")
                .merge(
                    df_join.alias("s"),
                    condicao_merge
                )
                .whenMatchedUpdateAll()
                .whenNotMatchedInsertAll()
                .execute()
            )

            print(f"✅ MERGE concluído com sucesso em {tabela_destino}")

        else:
            print(f"🆕 Tabela {tabela_destino} não existe — criando nova tabela...")

            df_join.write \
                .format("delta") \
                .partitionBy("data") \
                .mode("append") \
                .option("overwriteSchema", "true") \
                .saveAsTable(tabela_destino)

            print(f"✅ Tabela {tabela_destino} criada e dados inseridos com sucesso.")

    else:
        print("⚠️ Nenhum dado para atualizar.")
